# Importing Libraries

In [41]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

import json
import pickle
import numpy as np
import random

import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD

# Preparing The Data for Chatbot Training

In [42]:
words = []
classes = []
documents = []
ignore_letters = ['?', '!']

data_file = open('Chatbot_dataset.json').read()
intents = json.loads(data_file)

# Printing Dataset

In [43]:
intents

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi',
    'Hello',
    'Hey',
    'Good morning',
    'Good evening',
    'Greetings',
    'Hi assistant',
    'Hello there',
    'Hey there',
    'Good afternoon',
    'Is anyone available',
    'Hello bot',
    'Hi travel team',
    'Hey support',
    'Hello travel assistant'],
   'responses': ['Hello. I am your travel assistant. How can I help you today?']},
  {'tag': 'goodbye',
   'patterns': ['Bye',
    'Goodbye',
    'See you',
    'See you later',
    'Talk to you later',
    'Thanks bye',
    'Catch you later',
    'Have a nice day',
    'Thank you goodbye',
    'I am leaving',
    'Exit',
    'Bye for now',
    'See you soon',
    'Closing chat',
    'End conversation'],
   'responses': ['Thank you for choosing us. Have a safe journey.']},
  {'tag': 'identity',
   'patterns': ['Who are you',
    'What is your name',
    'Are you a human',
    'Are you a bot',
    'Tell me about yourself',
    'What can you do',
    'Introduce yo

# Tokenizing Patters and Creating the Corpus

In [44]:
nltk.download('punkt')
nltk.download('punkt_tab') 
nltk.download('omw-1.4')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [45]:
for intent in intents.get("intents", []):
    tag = intent.get("tag")
    
    for pattern in intent.get("patterns", []):
        tokens = nltk.word_tokenize(pattern.lower())
        
        tokens = [word for word in tokens if word not in ignore_letters]
        
        words.extend(tokens) 
        
        documents.append((tokens, tag))
        
        if tag not in classes:
            classes.append(tag)
            
print("Corpus Created Successfully")
print("Total Documents: ", len(documents))
print("Total Classes: ", len(classes))
print("Total Words: ", len(words))

Corpus Created Successfully
Total Documents:  300
Total Classes:  20
Total Words:  773


In [46]:
from nltk.corpus import wordnet as wn

synonyms = wn.synsets('travel')
print(synonyms[0].definition())

the act of going from one place to another


# Lemmatizing, Sorting and Saving Words and Classes


In [47]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_letters]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))

print(len(documents), "documents")
print(len(classes), "classes", classes)
print(len(words), "unique lemmatized words", words)

pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

300 documents
20 classes ['book_hotel', 'cancel_booking', 'check_availability', 'contact', 'flight_booking', 'goodbye', 'greeting', 'identity', 'offers', 'payment_methods', 'provide_budget', 'provide_destination', 'provide_travel_date', 'provide_travelers', 'refund_policy', 'safety_guidelines', 'transport', 'travel_packages', 'visa_information', 'weather']
292 unique lemmatized words ['1', '15th', '20000', '20th', '3', '4', '5', '50000', 'a', 'about', 'accept', 'accepted', 'accommodation', 'adult', 'adventure', 'affordable', 'after', 'afternoon', 'air', 'airline', 'airport', 'am', 'any', 'anyone', 'apply', 'april', 'are', 'arrange', 'arrangement', 'assistance', 'assistant', 'at', 'availability', 'available', 'back', 'bali', 'banking', 'best', 'book', 'booking', 'bot', 'budget', 'bus', 'by', 'bye', 'cab', 'call', 'can', 'cancel', 'cancellation', 'car', 'card', 'care', 'catch', 'charge', 'chat', 'cheap', 'check', 'child', 'climate', 'closing', 'code', 'complaint', 'condition', 'confirmed

# Creating and Preparing the Training Data

In [48]:
import numpy as np
import random
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:

    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
# output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

# shuffle our features and turn into np.array
# Shuffle training data before converting to numpy array
random.shuffle(training)

# Print the shapes of each element in the training list
for item in training:
    print("Shape of bag:", np.array(item[0]).shape)
    print("Shape of output_row:", np.array(item[1]).shape)

# Separate features (X) and labels (Y) from the training data
train_x = np.array([bag for bag, _ in training])
train_y = np.array([output_row for _, output_row in training])

print("Training data created")

Shape of bag: (292,)
Shape of output_row: (20,)
Shape of bag: (292,)
Shape of output_row: (20,)
Shape of bag: (292,)
Shape of output_row: (20,)
Shape of bag: (292,)
Shape of output_row: (20,)
Shape of bag: (292,)
Shape of output_row: (20,)
Shape of bag: (292,)
Shape of output_row: (20,)
Shape of bag: (292,)
Shape of output_row: (20,)
Shape of bag: (292,)
Shape of output_row: (20,)
Shape of bag: (292,)
Shape of output_row: (20,)
Shape of bag: (292,)
Shape of output_row: (20,)
Shape of bag: (292,)
Shape of output_row: (20,)
Shape of bag: (292,)
Shape of output_row: (20,)
Shape of bag: (292,)
Shape of output_row: (20,)
Shape of bag: (292,)
Shape of output_row: (20,)
Shape of bag: (292,)
Shape of output_row: (20,)
Shape of bag: (292,)
Shape of output_row: (20,)
Shape of bag: (292,)
Shape of output_row: (20,)
Shape of bag: (292,)
Shape of output_row: (20,)
Shape of bag: (292,)
Shape of output_row: (20,)
Shape of bag: (292,)
Shape of output_row: (20,)
Shape of bag: (292,)
Shape of output_row

In [49]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

from tensorflow.keras.optimizers import SGD 

sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

print("Model created")

Epoch 1/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 907us/step - accuracy: 0.0567 - loss: 2.9944     
Epoch 2/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 858us/step - accuracy: 0.0967 - loss: 2.9445
Epoch 3/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 850us/step - accuracy: 0.1733 - loss: 2.8393
Epoch 4/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 830us/step - accuracy: 0.2200 - loss: 2.6949
Epoch 5/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 832us/step - accuracy: 0.2867 - loss: 2.4754
Epoch 6/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 846us/step - accuracy: 0.3800 - loss: 2.1582
Epoch 7/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 841us/step - accuracy: 0.4433 - loss: 1.9487
Epoch 8/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 853us/step - accuracy: 0.5133 - loss: 1.7421
Epoch 9/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 837us/step - accuracy: 0.5333 - loss: 1.6183
Epoch 10/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 836us/step - accuracy: 0.5867 - loss: 1.3850
Epoch 11/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 858us/step - accuracy: 0.6400 - loss: 1.2120
Epoch 12/200
60/60 ━━━━━━━━━━━━━━━━━

In [50]:
model.save('Travo_Chatbot_model.h5')

# Loading the Trained Model and Data for Chatbot Inference

In [51]:
model = load_model('Travo_Chatbot_model.h5')

intents = json.loads(open('Chatbot_dataset.json').read())

words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))

# Functions for Cleaning, Tokenizing, and Predicting Intents

In [52]:
def clean_up_sentence(sentence):

    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)

    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=True):

    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)

    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words) 
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):

    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]

    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

# Functions for Generating Chatbot Responses

In [53]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag'] == tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res

# Graphical User Interface (GUI) for the Chatbot

In [ ]:
import tkinter as tk
from tkinter import scrolledtext

#  SEND FUNCTION 
def send_message(event=None):
    msg = entry_box.get().strip()
    entry_box.delete(0, tk.END)

    if msg:
        chat_log.config(state=tk.NORMAL)

        # User bubble
        chat_log.insert(tk.END, "You\n", "user_name")
        chat_log.insert(tk.END, msg + "\n\n", "user_msg")

        # Bot response
        response = chatbot_response(msg)
        chat_log.insert(tk.END, "Restrobot\n", "bot_name")
        chat_log.insert(tk.END, response + "\n\n", "bot_msg")

        chat_log.config(state=tk.DISABLED)
        chat_log.yview(tk.END)


# MAIN WINDOW
base = tk.Tk()
base.title(" TRAVO - AI Chatbot")
base.geometry("650x650")
base.configure(bg="#ecf0f1")
base.minsize(500, 550)

#  HEADER 
header = tk.Label(
    base,
    text=" TRAVO - Smart Travel & Hospitality Assistant",
    bg="#34495e",
    fg="white",
    font=("Segoe UI", 16, "bold"),
    pady=15
)
header.pack(fill=tk.X)

# CHAT AREA
chat_log = scrolledtext.ScrolledText(
    base,
    wrap=tk.WORD,
    state=tk.DISABLED,
    font=("Segoe UI", 11),
    bg="white",
    bd=0,
    padx=15,
    pady=15
)
chat_log.pack(padx=15, pady=10, fill=tk.BOTH, expand=True)

# TAG STYLING 
chat_log.tag_config("user_name", foreground="#1e88e5", font=("Segoe UI", 10, "bold"))
chat_log.tag_config("user_msg", background="#e3f2fd", lmargin1=50, lmargin2=50, rmargin=10)

chat_log.tag_config("bot_name", foreground="#2e7d32", font=("Segoe UI", 10, "bold"))
chat_log.tag_config("bot_msg", background="#e8f5e9", lmargin1=10, lmargin2=10, rmargin=50)

# WELCOME MESSAGE 
chat_log.config(state=tk.NORMAL)
chat_log.insert(tk.END, "TRAVO\n", "bot_name")
chat_log.insert(tk.END, "Hello! I'm TRAVO.\nHow can I help you today?\n\n", "bot_msg")
chat_log.config(state=tk.DISABLED)

#  INPUT FRAME 
input_frame = tk.Frame(base, bg="#ecf0f1")
input_frame.pack(fill=tk.X, padx=15, pady=10)

entry_box = tk.Entry(
    input_frame,
    font=("Segoe UI", 12),
    bd=0,
    relief=tk.FLAT
)
entry_box.pack(side=tk.LEFT, fill=tk.X, expand=True, ipady=8, padx=(0, 10))
entry_box.bind("<Return>", send_message)

send_button = tk.Button(
    input_frame,
    text="Send",
    font=("Segoe UI", 11, "bold"),
    bg="#2ecc71",
    fg="white",
    activebackground="#27ae60",
    bd=0,
    padx=20,
    pady=8,
    cursor="hand2",
    command=send_message
)
send_button.pack(side=tk.RIGHT)

# RUN APP 
base.mainloop()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
